## Random Sample Imputation

In this recipe, we will perform random sample imputation using pandas and Feature Engine.

In [1]:
import pandas as pd

# to split the data sets
from sklearn.model_selection import train_test_split

# to impute missing data with feature-engine
from feature_engine.missing_data_imputers import RandomSampleImputer

In [2]:
# load data
data = pd.read_csv('creditApprovalUCI.csv')
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,NaN,u,g,q,h,NaN,NaN,NaN,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# let's separate into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((483, 15), (207, 15))

In [4]:
# find the percentage of missing data within those variables

X_train.isnull().mean()

A1     0.008282
A2     0.022774
A3     0.140787
A4     0.008282
A5     0.008282
A6     0.008282
A7     0.008282
A8     0.140787
A9     0.140787
A10    0.140787
A11    0.000000
A12    0.000000
A13    0.000000
A14    0.014493
A15    0.000000
dtype: float64

## Random Sample imputation with pandas

In [5]:
# extract a random sample (as many values as missing values in the variable)

number_missing_values = X_train['A2'].isnull().sum()
number_missing_values

11

In [6]:
# extract a random sample (as many values as missing values in the variable)

random_sample_train = X_train['A2'].dropna().sample(number_missing_values, random_state=0)

In [7]:
# re-index the random sample so that we can join it to our original data

random_sample_train.index = X_train[X_train['A2'].isnull()].index

random_sample_train.index

Int64Index([97, 500, 329, 83, 254, 608, 445, 450, 515, 286, 86], dtype='int64')

In [8]:
# replace the missing values
X_train.loc[X_train['A2'].isnull(), 'A2'] = random_sample_train

X_train['A2'].isnull().sum()

0

In [9]:
# repeat in a loop for the rest of the variables
# and for both train and test set

for var in ['A1', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']:

    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(
        X_train[var].isnull().sum(), random_state=0)

    random_sample_test = X_train[var].dropna().sample(
        X_test[var].isnull().sum(), random_state=0)

    # re index the random sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA 
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    
# check missing data
X_train[['A1', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']].isnull().sum()

A1    0
A3    0
A4    0
A5    0
A6    0
A7    0
A8    0
dtype: int64

## Random Sample imputation with Feature Engine

In [10]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

In [11]:
# let's create a random sample imputer

imputer = RandomSampleImputer()

imputer.fit(X_train)

RandomSampleImputer(random_state=None, seed='general', seeding_method='add',
                    variables=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8',
                               'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15'])

In [12]:
# the imputer stores the train set

imputer.X.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,v,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,v,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,6,t,g,360.0,1332


In [13]:
# transform the data - replace the missing values

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

C:\Users\Sole\Anaconda3\envs\feml\lib\site-packages\sklearn\utils\validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)


In [14]:
# check that null values were replaced
X_train.isnull().mean()

A1     0.0
A2     0.0
A3     0.0
A4     0.0
A5     0.0
A6     0.0
A7     0.0
A8     0.0
A9     0.0
A10    0.0
A11    0.0
A12    0.0
A13    0.0
A14    0.0
A15    0.0
dtype: float64

## Random Sampling seeding on variable values

In [15]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

In [16]:
imputer_obs = RandomSampleImputer(random_state=['A8', 'A3'], seed='observation', seeding_method='add')

In [17]:
imputer_obs.fit(X_train)

RandomSampleImputer(random_state=['A8', 'A3'], seed='observation',
                    seeding_method='add',
                    variables=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8',
                               'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15'])

In [18]:
X_train_tt = imputer_obs.transform(X_train)
X_test_tt = imputer_obs.transform(X_test)